In [4]:
import json
import re
import pandas as pd

# --- 1) Fil-stier (ret hvis nødvendigt) ---
ACCORDION_PATH = "accordion_response_shots.json"
BETBUILDER_PATH = "betbuilder_price_response_shots.json"  # bruges kun til combo-odds, ikke nødvendigt for player-odds

# --- 2) Load JSON ---
with open(ACCORDION_PATH, "r", encoding="utf-8") as f:
    accordion = json.load(f)

with open(BETBUILDER_PATH, "r", encoding="utf-8") as f:
    betbuilder = json.load(f)

# --- 3) Hjælpefunktioner ---
def parse_line_from_marketid(market_id: str):
    """
    Marked-id ser typisk sådan ud:
    m-...-PLYPROPFOU-0.5-401328
    Vi tager tallet før sidste '-' som line (0.5, 1.5, 2.5 ...).
    """
    if not isinstance(market_id, str):
        return None
    m = re.search(r"-([0-9]+(?:\.[0-9]+)?)-\d+$", market_id)
    return float(m.group(1)) if m else None

def selection_decimal(sel: dict):
    """
    Odds ligger ofte i sel["marketSelectionPriceFormats"]["1"] (decimal odds som string),
    eller i sel["oddsFormats"]["decimal"].
    """
    mp = sel.get("marketSelectionPriceFormats") or sel.get("oddsFormats") or {}
    if isinstance(mp, dict):
        if "1" in mp:
            return mp["1"]
        if "decimal" in mp:
            return mp["decimal"]
    return None

def extract_player_and_prop(market: dict):
    """
    I accordion markets ligger player ofte i:
    market["marketSpecifics"]["groupLabels"]["2"]
    og prop i:
    market["marketSpecifics"]["groupLabels"]["1"]

    Fallback: parse fra market["label"] med "prop | player".
    """
    group_labels = (market.get("marketSpecifics", {}) or {}).get("groupLabels", {}) or {}
    player = group_labels.get("2")
    prop = group_labels.get("1")

    if not player or not prop:
        lab = market.get("label") or market.get("marketFriendlyName") or ""
        if "|" in lab:
            left, right = lab.split("|", 1)
            prop = prop or left.strip()
            player = player or right.strip()

    return player, prop

# --- 4) Parse accordion til to tabeller: markets og selections ---
accordions = accordion.get("data", {}).get("accordions", {})

market_rows = []
selection_rows = []

for _, group in accordions.items():
    # markets
    for m in group.get("markets", []):
        player, prop = extract_player_and_prop(m)
        market_id = m.get("id")
        market_rows.append({
            "eventId": m.get("eventId"),
            "marketId": market_id,
            "marketTemplateId": m.get("marketTemplateId"),
            "marketLabel": m.get("label"),
            "marketFriendlyName": m.get("marketFriendlyName"),
            "status_market": m.get("status"),
            "deadline": m.get("deadline"),
            "player": player,
            "prop": prop,
            "line": parse_line_from_marketid(market_id),
        })

    # selections (odds)
    for s in group.get("selections", []):
        selection_rows.append({
            "selectionId": s.get("id"),
            "marketId": s.get("marketId"),
            "selectionLabel": s.get("label"),     # fx "Over 0.5"
            "odds_decimal": selection_decimal(s), # fx "1.22"
            "status_selection": s.get("status"),
            "isDisplayed": s.get("isDisplayed"),
            "isBoosted": s.get("isBoosted"),
        })

df_markets = pd.DataFrame(market_rows)
df_selections = pd.DataFrame(selection_rows)

# --- 5) Merge til samlet tabel ---
df_all = df_selections.merge(df_markets, on="marketId", how="left")

# Konverter odds til float når muligt
df_all["odds_decimal"] = pd.to_numeric(df_all["odds_decimal"], errors="coerce")

# --- 6) Eksempler på outputs ---
print("Rækker i df_all:", len(df_all))
print(df_all[["player", "prop", "line", "selectionLabel", "odds_decimal"]].head(10))

# A) Kun "Over 0.5"
df_over_05 = df_all[
    (df_all["selectionLabel"].astype(str).str.strip() == "Over 0.5")
].sort_values(["player"])

print("\nKun Over 0.5 (første 20):")
print(df_over_05[["player", "prop", "line", "selectionLabel", "odds_decimal"]].head(20))

# B) Find specifik spiller (Alexis Mac Allister)
df_alexis = df_over_05[df_over_05["player"].astype(str).str.contains("Alexis Mac Allister", na=False)]
print("\nAlexis Mac Allister Over 0.5:")
print(df_alexis[["player", "prop", "line", "selectionLabel", "odds_decimal", "marketId", "selectionId"]])

COLUMNS = [
    "selectionLabel",
    "odds_decimal",
    "status_selection",
    "marketLabel",
    "deadline",
    "player",
    "line",
]

df_all_clean = df_all[COLUMNS].copy()
df_over_05_clean = df_over_05[COLUMNS].copy()

# --- 10) Gem til CSV ---
df_all_clean.to_csv("player_shots.csv", index=False, encoding="utf-8")


# --- 8) Betbuilder (valgfrit): combo-odds fra din betbuilder fil ---
# Bemærk: din betbuilder response indeholder kun den samlede combo-odds (fx 1.56),
# ikke odds per player-market. Derfor bruger vi accordion til player-odds.
combo_prices = betbuilder.get("betBuilderPrices", [])
if combo_prices:
    df_combo = pd.DataFrame([{
        "fixtureId": x.get("fixtureId"),
        "combo_odds_decimal": (x.get("oddsFormats", {}) or {}).get("decimal"),
    } for x in combo_prices])
    print("\nBetbuilder combo-odds (hvis tilgængelig):")
    print(df_combo)

Rækker i df_all: 150
               player                         prop  line selectionLabel  \
0         Rio Ngumoha  Spillers samlede antal skud  None       Over 2.5   
1         Rio Ngumoha  Spillers samlede antal skud  None       Over 0.5   
2         Rio Ngumoha  Spillers samlede antal skud  None       Over 1.5   
3         Declan Rice  Spillers samlede antal skud  None       Over 0.5   
4         Declan Rice  Spillers samlede antal skud  None       Over 1.5   
5         Declan Rice  Spillers samlede antal skud  None       Over 2.5   
6    Leandro Trossard  Spillers samlede antal skud  None       Over 3.5   
7    Leandro Trossard  Spillers samlede antal skud  None       Over 1.5   
8    Leandro Trossard  Spillers samlede antal skud  None       Over 2.5   
9  Christian Nørgaard  Spillers samlede antal skud  None       Over 1.5   

   odds_decimal  
0          6.00  
1          1.38  
2          2.70  
3          1.16  
4          1.78  
5          3.20  
6          3.70  
7        